# Step0: Install environment

In [2]:
import vapoursynth as vs
import havsfunc as haf
import mvsfunc as mvf
from vapoursynth import core
import awsmfunc as awsf

# Step1: Select BDMV Largest M2TS and its MLPS

# Step2: BDMV to MKV

In [86]:
get_captures("REMUX", MOVIE)

Capture REMUX


# Step3: Transcode

In [51]:
import os, pyperclip, re, subprocess
from pymediainfo import MediaInfo
from datetime import datetime
crf=17.0
qcomp=0.7
aq_mode=3
aq_s=0.7
bitdepth=8
MOVIE=r"C:\Users\heber\Music\A Gentleman.2017.1080p.BluRay.REMUX.AVC.DTS-louco@PTH.mkv"

## H264

In [3]:
newline="\r\n"
remotecwd="/media/lind/SSD/compressor"
remotehost="ubuntu20local"

x264_stmt=f"x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --bframes 11 --me umh  --subme 11 \
--merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 \
--vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 \
--min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv -"

args=x264_stmt.replace(" --","\n --").split("\n")[1:]
args=sorted(args)
print("\n".join(["x264 参数："]+args))

remote_x264stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0|python {os. getcwd()}\\aio_buffer.py|\
ssh {remotehost} "python3 {remotecwd}/aio_buffer.py|zstdcat|~/anaconda3/bin/{x264_stmt} 2>&1 | tee x264.log"{newline}'

print("==========\n远程执行编码 x264:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd+=f'set TEST=YES{newline}'
cmd+=remote_x264stmt
cmd+=f'scp seclab:~/264.mkv {os. getcwd()}\\264.mkv{newline}'
cmd+=f'scp seclab:~/264.mkv D:\\BT\\samples\\264.mkv{newline}'
cmd+=f'pause{newline}'
pyperclip.copy(cmd)
print(cmd)

x264 参数：
 --aq-mode 3
 --aq-strength 0.7
 --b-adapt 2
 --bframes 11
 --chroma-qp-offset -1
 --crf 17.0
 --deblock -3:-3
 --demuxer y4m
 --direct auto
 --keyint 240
 --level 4.1
 --me umh 
 --merange 48
 --min-keyint 1
 --no-dct-decimate
 --no-fast-pskip
 --no-mbtree
 --output 264.mkv -
 --preset slower
 --profile high
 --psy-rd 1.00:0.00
 --qcomp 0.7
 --rc-lookahead 72
 --ref 4
 --subme 11
 --trellis 2
 --vbv-bufsize 78125
 --vbv-maxrate 62500
远程执行编码 x264:
set MOVIE=C:\Users\heber\Music\A Gentleman.2017.1080p.BluRay.REMUX.AVC-louco.mkv
set TEST=YES
vspipe.exe --y4m "C:\Users\heber\Documents\Compressor\vs.py" - |zstd -0|python C:\Users\heber\Documents\Compressor\aio_buffer.py|ssh ubuntu20local "python3 /media/lind/SSD/compressor/aio_buffer.py|zstdcat|~/anaconda3/bin/x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 --crf 17.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --bframes 11 --me umh  --subme 11 --merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-

## H265

In [626]:
x265_stmt=f"x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} \
--aq-strength {aq_s} --rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 \
--qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-opt-qp-pps \
--no-opt-ref-list-length-pps --aud --repeat-headers --me 3 --subme 5 --merange 48 --b-intra --limit-tu 0 --no-rect \
--no-amp --no-open-gop --pools + --keyint 240 --min-keyint 1 --bframes 8 --max-merge 4 --ref 4 --weightb --rc-lookahead 72 \
--scenecut 40 --no-strong-intra-smoothing --vbv-bufsize 160000 --vbv-maxrate 160000 --input-depth 10 --output 265.mkv -"
#--input-depth 10 

remote_x265stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |pigz --fast|python {os. getcwd()}\\aio_buffer.py|\
ssh seclab "python3 {remotecwd}/aio_buffer.py|pigz -dc|{x265_stmt} 2>&1 | tee x265.log"{newline}'

# remote_x265stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |{x265_stmt}"{newline}'

args=x265_stmt.replace(" --","\n --").split("\n")[1:]
args=sorted(args)
print("\n".join(["x265 参数："]+args))

print("==========\n远程执行编码 x265:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd+=remote_x265stmt
cmd+=f'scp seclab:~/265.mkv {os. getcwd()}\\265.mkv{newline}'
cmd+=f'scp seclab:~/265.mkv D:\\BT\\samples\\265.mkv{newline}'
cmd+=f'pause{newline}'
pyperclip.copy(cmd)
print(cmd)

x265 参数：
 --aq-mode 1
 --aq-strength 0.6
 --aud
 --b-intra
 --bframes 8
 --cbqpoffs -1
 --crf 20.0
 --crqpoffs -3
 --ctu 32
 --deblock -1:-1
 --high-tier
 --input-depth 10
 --keyint 240
 --level-idc 5.1
 --limit-tu 0
 --max-merge 4
 --me 3
 --merange 48
 --min-keyint 1
 --no-amp
 --no-open-gop
 --no-opt-qp-pps
 --no-opt-ref-list-length-pps
 --no-rect
 --no-sao
 --no-sao-non-deblock
 --no-strong-intra-smoothing
 --output 265.mkv -
 --pools +
 --preset slower
 --profile main10
 --psy-rd 2.0
 --psy-rdoq 1.0
 --qcomp 0.7
 --qg-size 8
 --rc-lookahead 72
 --rd 4
 --rdoq-level 2
 --ref 4
 --repeat-headers
 --scenecut 40
 --selective-sao 0
 --subme 5
 --tu-inter-depth 4
 --tu-intra-depth 4
 --vbv-bufsize 160000
 --vbv-maxrate 160000
 --weightb
 --y4m -D 10
远程执行编码 x265:
set MOVIE=C:\Users\heber\Music\The.Apartment.1960.Remastered.1080p.GBR.Blu-ray.AVC.DTS-HD.MA.5.1-ShawnQiang@PTHome.mkv
vspipe.exe --y4m "C:\Users\heber\Documents\Compressor\vs.py" - |pigz --fast|python C:\Users\heber\Documents\C

## H264&H265

In [5]:
x264_stmt=f"x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --bframes 11 --me umh  --subme 11 \
--merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 \
--vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 \
--min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv -"

x265_stmt=f"x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} \
--aq-strength {aq_s} --rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 \
--qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-opt-qp-pps \
--no-opt-ref-list-length-pps --aud --repeat-headers --me 3 --subme 5 --merange 48 --b-intra --limit-tu 0 --no-rect \
--no-amp --no-open-gop --pools + --keyint 240 --min-keyint 1 --bframes 8 --max-merge 4 --ref 4 --weightb --rc-lookahead 72 \
--scenecut 40 --no-strong-intra-smoothing --vbv-bufsize 160000 --vbv-maxrate 160000 --input-depth 10 --output 265.mkv -"


remote_stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0 -T4|python {os. getcwd()}\\aio_buffer.py|\
ssh {remotehost} "python3 {remotecwd}/aio_buffer.py|zstdcat| tee \
>({x264_stmt} 2>&1 | tee x264.log) \
>({x265_stmt} 2>&1 | tee x265.log) \
>/dev/null{newline}'


print("==========\n远程执行编码 x264+x265:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd=f'set TEST=NO{newline}'
cmd+=remote_stmt
cmd+=f'scp {remotehost}:~/264.mkv {os. getcwd()}\\264.mkv{newline}'
cmd+=f'scp {remotehost}:~/265.mkv {os. getcwd()}\\265.mkv{newline}'

cmd+=f'ssh {remotehost} "cat x264.log|sed \\"s/.*\\r//g\\" " >{os. getcwd()}\\x264.log & '
cmd+=f'ssh {remotehost} "cat x265.log|sed \\"s/.*\\r//g\\" " >{os. getcwd()}\\x265.log{newline}'
cmd+=f'pause{newline}'
pyperclip.copy(cmd)
print(cmd)

远程执行编码 x264+x265:
set TEST=NO
vspipe.exe --y4m "C:\Users\heber\Documents\Compressor\vs.py" - |zstd -0 -T4|python C:\Users\heber\Documents\Compressor\aio_buffer.py|ssh ubuntu20local "python3 /media/lind/SSD/compressor/aio_buffer.py|zstdcat| tee >(x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 --crf 17.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --bframes 11 --me umh  --subme 11 --merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 --vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 --min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv - 2>&1 | tee x264.log) >(x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf 17.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 --qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --

# Step4: Remux MKV
## x264

In [75]:
from pymediainfo import MediaInfo
from datetime import datetime
import os

class nullabledict:
    def __init__(self, input_dict):
        self.data=input_dict
    def __getitem__(self, key):
        if key in self.data:
            return self.data[key]
        else: return "?"
    def __contains__(self, key):
        if key in self.data: return True
        else : return False

def generate_videoinfo(filename, logname="", src=""):
    media_info = MediaInfo.parse(filename)
    for t in media_info.video_tracks:
        props=nullabledict(t.to_data())
#         for i in props.items(): print(i)
        video_codec=f"{props['encoded_library_name']} {props['format_profile']} @ {str(round(props['bit_rate'] / (1024 * 1024), 2))}Mb/s"
        runtime=props['other_duration'][0]
        resolution=f"{props['sampled_width']}x{props['sampled_height']}"
        display_aspect_ratio=props['display_aspect_ratio']
        frame_rate=props["frame_rate"]
        bit_depth=props["other_bit_depth"][0]
    audio_codec=[]
    for t in media_info.audio_tracks:
        props=nullabledict(t.to_data())
#         for i in t.to_data().items(): print(i)
        chns=props['channel_s']
        if chns==6: chns="5.1"
        dtsma=""
        if props['commercial_name']=='DTS-HD Master Audio':dtsma="-HD MA"
        audio_codec.append(f"{props['other_language'][0]} {props['format']}{dtsma} {chns}\
 @ {str(round(props['bit_rate'] / (1024), 2))}Kb/s")
    audioinfo=""    
    for ac in audio_codec:
        audioinfo+=f'AUDiO CODEC...............: {ac} \n'
    chscnt=0
    subtitledesc=''
    for t in media_info.text_tracks:
        props=nullabledict(t.to_data())
    #     for i in props.items(): print(i)
        if 'title' in props:
            title=props['title']
        else: title=props['language']
        
        if title=='zh': 
            title='chs';chscnt+=1
            if chscnt>2: title='cht'
            if chscnt%2==0: title+='&eng'
        subtitlefmt=f"{props['codec_id']} : {props['other_language'][0]}({title})"
        subtitledesc+=f"SUBTiTLES.................: {subtitlefmt}\n"
    NEWMOVIE_size=str(round(os.path.getsize(filename) / (1024 * 1024 * 1024), 3)) + ' GB'
    MOVIE_filename="".join(re.findall(r'([^\\]*$)', MOVIE))
    RELEASE_filename="".join(re.findall(r"([^\\]*$)", filename.replace(".mkv","")))
    if len(src)>0: MOVIE_filename="".join(re.findall(r'([^\\]*$)', src))
    
    transcodelog=""
    if len(logname)>0:
        transcodelog=f'.{logname.replace(".log","")}.Info\n'
        with open(logname) as f:
            lines=f.readlines()
            for line in lines:
                if '[info]' in line:
                    if 'Avg QP' in line or 'profile' in line or 'rames:' in line:
                        transcodelog+=line
    release_info=f'''[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: {RELEASE_filename}
RELEASE DATE..............: {datetime.today().strftime('%Y-%m-%d')}
RELEASE SiZE..............: {NEWMOVIE_size}
SOURCE....................: {MOVIE_filename.replace(".mkv","")}

.Media.Info
RUNTiME...................: {runtime}
ViDEO CODEC...............: {video_codec}
RESOLUTiON................: {resolution}
DiSPLAY ASPECT RATiO......: {display_aspect_ratio}
FRAME RATE................: {frame_rate}
BIT DEPTH.................: {bit_depth}
{audioinfo} 
{subtitledesc}

{transcodelog}[/quote]'''
    print(release_info)
    pyperclip.copy(release_info)
    return release_info
    
def get_captures(codec, filename):
    if "\\" not in filename:
        filename=f"{os.getcwd()}\\{filename}"
    if not os.path.exists(f"{codec}_captures"):
        os.mkdir(f"{codec}_captures")
    ffmpeg_extract_cmd=''
    for i in range(0,12):
        time=f'0{int(i/6)}:{i%6}2'
        ffmpeg_extract_cmd+=f'ffmpeg -y -ss {time}:00 -i "{filename}" -vframes 1\
        -q:v 2 {os.getcwd()}\\{codec}_captures\\{codec}_capture_{i*2}.jpg & '
        time=f'0{int(i/6)}:{i%6}7'
        ffmpeg_extract_cmd+=f'ffmpeg -y -ss {time}:00 -i "{filename}" -vframes 1\
        -q:v 2 {os.getcwd()}\\{codec}_captures\\{codec}_capture_{i*2+1}.jpg & '
    pyperclip.copy(ffmpeg_extract_cmd)
    ffmpeg_extract_cmd+="pause"
    subprocess.run(f"start cmd /c {ffmpeg_extract_cmd} ^& pause", shell=True)
    print("Capture",codec)
    
def get_mediainfo(filename):
    nfo=subprocess.check_output(f'mediainfo\mediainfo.exe "{filename}"').decode("utf-8")
    pyperclip.copy(f'[hide][font=Lucida Console]\n{nfo}[/hide]')
    print("Copy Mediainfo")
    return f'[hide][font=Lucida Console]\n{nfo}[/hide]'

In [52]:
# title="".join(re.findall(r'([^\\]*\d{4}p)', MOVIE))
# NEWMOVIE=f'{os.getcwd()}\\{title}.BluRay.x264.DTS-PTH.mkv'
x264_filename="".join(re.findall(r'([^\\]*$)', MOVIE)).replace("REMUX.AVC", "x264")
NEWMOVIE=f'{os.getcwd()}\\{x264_filename}'
print(x264_filename)

cmd=f'{os. getcwd()}\\mkvtoolnix\\mkvmerge.exe -o "{NEWMOVIE}" -D ( "{MOVIE}" )\
    -A -S -T -M -B --no-chapters ( "{os. getcwd()}\\264.mkv" )'
pyperclip.copy(cmd)
print(f"即将执行: \n   [剩] {MOVIE}\n + [视] {os. getcwd()}\\264.mkv\n-> [全] {NEWMOVIE}\n=================")
print(cmd)

A Gentleman.2017.1080p.BluRay.x264.DTS-louco@PTH.mkv
即将执行: 
   [剩] C:\Users\heber\Music\A Gentleman.2017.1080p.BluRay.REMUX.AVC.DTS-louco@PTH.mkv
 + [视] C:\Users\heber\Documents\Compressor\264.mkv
-> [全] C:\Users\heber\Documents\Compressor\A Gentleman.2017.1080p.BluRay.x264.DTS-louco@PTH.mkv
C:\Users\heber\Documents\Compressor\mkvtoolnix\mkvmerge.exe -o "C:\Users\heber\Documents\Compressor\A Gentleman.2017.1080p.BluRay.x264.DTS-louco@PTH.mkv" -D ( "C:\Users\heber\Music\A Gentleman.2017.1080p.BluRay.REMUX.AVC.DTS-louco@PTH.mkv" )    -A -S -T -M -B --no-chapters ( "C:\Users\heber\Documents\Compressor\264.mkv" )


In [53]:
subprocess.run(f"start cmd /c {cmd} ^& pause", shell=True)
None

In [54]:
generate_videoinfo(x264_filename,"x264.log")
None

[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: A Gentleman.2017.1080p.BluRay.x264.DTS-louco@PTH
RELEASE DATE..............: 2020-12-15
RELEASE SiZE..............: 14.613 GB
SOURCE....................: A Gentleman.2017.1080p.BluRay.REMUX.AVC.DTS-louco@PTH

.Media.Info
RUNTiME...................: 2 h 12 min
ViDEO CODEC...............: x264 High@L4.1 @ 13.61Mb/s
RESOLUTiON................: 1920x800
DiSPLAY ASPECT RATiO......: 2.400
FRAME RATE................: 24.000
BIT DEPTH.................: 8 bits
AUDiO CODEC...............: Hindi DTS 5.1 @ 1473.63Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : English(en)
SUBTiTLES.................: S_TEXT/ASS : Chinese(cht&eng)
SUBTiTLES.................: S_TEXT/ASS : Chinese(cht)
SUBTiTLES.................: S_TEXT/ASS : Chinese(chs&eng)
SUBTiTLES.................: S_TEXT/ASS : Chinese(chs)


.x264.Info
x264 [info]: profile High, level 4.1
x264 [info]: frame I:2045  Avg QP:14.84  size:254902
x2

In [21]:
get_captures("x264", x264_filename)

Capture x264


In [25]:
get_mediainfo(x264_filename)
None

Copy Mediainfo


## x265

In [55]:
x265_filename="".join(re.findall(r'([^\\]*$)', MOVIE)).replace("REMUX.AVC", "x265.10bit")
NEWMOVIE=f'{os.getcwd()}\\{x265_filename}'

cmd=f'{os. getcwd()}\\mkvtoolnix\\mkvmerge.exe -o "{NEWMOVIE}" -D ( "{MOVIE}" )\
    -A -S -T -M -B --no-chapters ( "{os. getcwd()}\\265.mkv" )'
pyperclip.copy(cmd)
print(f"即将执行: \n   [剩] {MOVIE}\n + [视] {os. getcwd()}\\265.mkv\n-> [全] {NEWMOVIE}\n=================")
print(cmd)

即将执行: 
   [剩] C:\Users\heber\Music\A Gentleman.2017.1080p.BluRay.REMUX.AVC.DTS-louco@PTH.mkv
 + [视] C:\Users\heber\Documents\Compressor\265.mkv
-> [全] C:\Users\heber\Documents\Compressor\A Gentleman.2017.1080p.BluRay.x265.10bit.DTS-louco@PTH.mkv
C:\Users\heber\Documents\Compressor\mkvtoolnix\mkvmerge.exe -o "C:\Users\heber\Documents\Compressor\A Gentleman.2017.1080p.BluRay.x265.10bit.DTS-louco@PTH.mkv" -D ( "C:\Users\heber\Music\A Gentleman.2017.1080p.BluRay.REMUX.AVC.DTS-louco@PTH.mkv" )    -A -S -T -M -B --no-chapters ( "C:\Users\heber\Documents\Compressor\265.mkv" )


In [56]:
subprocess.run(f"start cmd /c {cmd} ^& pause", shell=True)

CompletedProcess(args='start cmd /c C:\\Users\\heber\\Documents\\Compressor\\mkvtoolnix\\mkvmerge.exe -o "C:\\Users\\heber\\Documents\\Compressor\\A Gentleman.2017.1080p.BluRay.x265.10bit.DTS-louco@PTH.mkv" -D ( "C:\\Users\\heber\\Music\\A Gentleman.2017.1080p.BluRay.REMUX.AVC.DTS-louco@PTH.mkv" )    -A -S -T -M -B --no-chapters ( "C:\\Users\\heber\\Documents\\Compressor\\265.mkv" ) ^& pause', returncode=0)

In [32]:
generate_videoinfo(x265_filename,"x265.log")
None

[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: A Gentleman.2017.1080p.BluRay.x265.10bit-louco@PTH
RELEASE DATE..............: 2020-12-15
RELEASE SiZE..............: 10.446 GB
SOURCE....................: A Gentleman.2017.1080p.BluRay.REMUX.AVC-louco@PTH

.Media.Info
RUNTiME...................: 2 h 12 min
ViDEO CODEC...............: x265 Main 10@L5.1@High @ 9.31Mb/s
RESOLUTiON................: 1920x800
DiSPLAY ASPECT RATiO......: 2.400
FRAME RATE................: 24.000
BIT DEPTH.................: 10 bits
AUDiO CODEC...............: Hindi DTS 5.1 @ 1473.63Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : English(en)
SUBTiTLES.................: S_TEXT/ASS : Chinese(cht&eng)
SUBTiTLES.................: S_TEXT/ASS : Chinese(cht)
SUBTiTLES.................: S_TEXT/ASS : Chinese(chs&eng)
SUBTiTLES.................: S_TEXT/ASS : Chinese(chs)


.x265.Info
[/quote]


In [33]:
get_captures("x265", x265_filename)

Capture x265


In [34]:
get_mediainfo(x265_filename)
None

Copy Mediainfo


# Step5 小修小补

## 音频转码为FLAC x264

In [597]:
# filename=x264_filename
def getcmd_convertflac(filename):
    cmds=[]
    media_info = MediaInfo.parse(filename)
    for i in media_info.audio_tracks:
        datas=i.to_data()
        lang=datas['language']
        audio_trackno=int(datas["stream_identifier"])+1
        if "streamorder" in datas:
#             print(datas)
            cmds.append(f'{os.getcwd()}\\mkvtoolnix\\mkvextract.exe {os.getcwd()}\\{filename}\
     tracks {datas["streamorder"]}:{os.getcwd()}\\audio')
            cmds.append(f'{os.getcwd()}\\eac3to\\eac3to.exe {os.getcwd()}\\audio {os.getcwd()}\\audio.flac')

            oldname=filename.split(".")
            for idx,part in enumerate(oldname):
                if datas['other_format'][0] in part:
                    oldname[idx]='FLAC'
            newname=".".join(oldname)

            cmds.append(f'{os. getcwd()}\\mkvtoolnix\\mkvmerge.exe -o "{os. getcwd()}\\{newname}" -A\
 ( "{os. getcwd()}\\{filename}" ) -D -S -T -M -B --no-chapters ( "{os. getcwd()}\\audio.flac" )')
            cmds.append(f'{os.getcwd()}\\mkvtoolnix\\mkvpropedit.exe "{os.getcwd()}\\{newname}" \
--edit track:a{audio_trackno} --set language={lang}')
            
    print(f"即将执行:\n [A{audio_trackno}] {filename}\n->    audio [{lang}]\n=eac3to=> audio.flac [{lang}]\n +[V] {filename}\n->    {newname}")
    print("=============")
    cmd=" ^& ".join(cmds)
    print(cmd)
    return cmd, newname, lang

In [598]:
cmd, x264_filename_new, lang=getcmd_convertflac(x264_filename)

即将执行:
 [A1] La.grande.bouffe.1973.1080p.Blu-ray.x264.LPCM.1.0-PTH.mkv
->    audio [fr]
=eac3to=> audio.flac [fr]
 +[V] La.grande.bouffe.1973.1080p.Blu-ray.x264.LPCM.1.0-PTH.mkv
->    La.grande.bouffe.1973.1080p.Blu-ray.x264.FLAC.1.0-PTH.mkv
C:\Users\heber\Documents\Compressor\mkvtoolnix\mkvextract.exe C:\Users\heber\Documents\Compressor\La.grande.bouffe.1973.1080p.Blu-ray.x264.LPCM.1.0-PTH.mkv     tracks 0:C:\Users\heber\Documents\Compressor\audio ^& C:\Users\heber\Documents\Compressor\eac3to\eac3to.exe C:\Users\heber\Documents\Compressor\audio C:\Users\heber\Documents\Compressor\audio.flac ^& C:\Users\heber\Documents\Compressor\mkvtoolnix\mkvmerge.exe -o "C:\Users\heber\Documents\Compressor\La.grande.bouffe.1973.1080p.Blu-ray.x264.FLAC.1.0-PTH.mkv" -A ( "C:\Users\heber\Documents\Compressor\La.grande.bouffe.1973.1080p.Blu-ray.x264.LPCM.1.0-PTH.mkv" ) -D -S -T -M -B --no-chapters ( "C:\Users\heber\Documents\Compressor\audio.flac" ) ^& C:\Users\heber\Documents\Compressor\mkvtoolnix\mkvpr

In [599]:
subprocess.run(f"start cmd /c {cmd} ^& pause", shell=True) 

CompletedProcess(args='start cmd /c C:\\Users\\heber\\Documents\\Compressor\\mkvtoolnix\\mkvextract.exe C:\\Users\\heber\\Documents\\Compressor\\La.grande.bouffe.1973.1080p.Blu-ray.x264.LPCM.1.0-PTH.mkv     tracks 0:C:\\Users\\heber\\Documents\\Compressor\\audio ^& C:\\Users\\heber\\Documents\\Compressor\\eac3to\\eac3to.exe C:\\Users\\heber\\Documents\\Compressor\\audio C:\\Users\\heber\\Documents\\Compressor\\audio.flac ^& C:\\Users\\heber\\Documents\\Compressor\\mkvtoolnix\\mkvmerge.exe -o "C:\\Users\\heber\\Documents\\Compressor\\La.grande.bouffe.1973.1080p.Blu-ray.x264.FLAC.1.0-PTH.mkv" -A ( "C:\\Users\\heber\\Documents\\Compressor\\La.grande.bouffe.1973.1080p.Blu-ray.x264.LPCM.1.0-PTH.mkv" ) -D -S -T -M -B --no-chapters ( "C:\\Users\\heber\\Documents\\Compressor\\audio.flac" ) ^& C:\\Users\\heber\\Documents\\Compressor\\mkvtoolnix\\mkvpropedit.exe "C:\\Users\\heber\\Documents\\Compressor\\La.grande.bouffe.1973.1080p.Blu-ray.x264.FLAC.1.0-PTH.mkv" --edit track:a1 --set language=fr 

## 音频转码为FLAC x265

In [600]:
cmd, x265_filename_new, lang=getcmd_convertflac(x265_filename)

即将执行:
 [A1] La.grande.bouffe.1973.1080p.Blu-ray.x265.10bit.LPCM.1.0-PTH.mkv
->    audio [fr]
=eac3to=> audio.flac [fr]
 +[V] La.grande.bouffe.1973.1080p.Blu-ray.x265.10bit.LPCM.1.0-PTH.mkv
->    La.grande.bouffe.1973.1080p.Blu-ray.x265.10bit.FLAC.1.0-PTH.mkv
C:\Users\heber\Documents\Compressor\mkvtoolnix\mkvextract.exe C:\Users\heber\Documents\Compressor\La.grande.bouffe.1973.1080p.Blu-ray.x265.10bit.LPCM.1.0-PTH.mkv     tracks 0:C:\Users\heber\Documents\Compressor\audio ^& C:\Users\heber\Documents\Compressor\eac3to\eac3to.exe C:\Users\heber\Documents\Compressor\audio C:\Users\heber\Documents\Compressor\audio.flac ^& C:\Users\heber\Documents\Compressor\mkvtoolnix\mkvmerge.exe -o "C:\Users\heber\Documents\Compressor\La.grande.bouffe.1973.1080p.Blu-ray.x265.10bit.FLAC.1.0-PTH.mkv" -A ( "C:\Users\heber\Documents\Compressor\La.grande.bouffe.1973.1080p.Blu-ray.x265.10bit.LPCM.1.0-PTH.mkv" ) -D -S -T -M -B --no-chapters ( "C:\Users\heber\Documents\Compressor\audio.flac" ) ^& C:\Users\heber\D

In [47]:
subprocess.run(f"start cmd /c {cmd} ^& pause", shell=True) 
None

## 修改字幕标题和顺序

In [76]:
def improve_subtitle(filename):
    filename=os.getcwd()+"\\"+filename
    edittable={}
    mkvpropeditcmd=""

    print("即将执行:")
    print(filename)
    textsort=[]#(textid, position)
    media_info = MediaInfo.parse(filename)
    for t in media_info.text_tracks:
        datas=dict(t.to_data())
#         print(datas)
        title=datas['other_language'][0]
        if "title" in datas:
            lang=None
            title=datas["title"]
            
            if "简" in datas["title"] or datas["title"]=="Chinese": 
                lang="CHS"
            if "繁" in datas["title"]: 
                lang="CHT"
            if "chs" in datas["title"].lower() or lang=="CHS":
                textsort.append((datas['stream_identifier']+1, 0, title))
            elif "cht" in datas["title"].lower() or lang=="CHT":
                textsort.append((datas['stream_identifier']+1, 1, title))
            else:
                textsort.append((datas['stream_identifier']+1, 2, title))
            if lang is not None:
                mkvpropeditcmd+=f'--edit track:s{datas["stream_identifier"]+1} --set name={lang} '
                print(f'   字幕轨{datas["stream_identifier"]+1}: [{datas["title"]}] => [{lang}]')
        elif datas["language"]=="zh":
            textsort.append((datas['stream_identifier']+1, 1, title))
        else :
            textsort.append((datas['stream_identifier']+1, 2, title))


    newpos=1
    textsortcmd=[]
    for textid, pos, title in sorted(textsort, key=lambda x: x[1]):
        if newpos==1: textsortcmd.append(f'--edit track:s{textid} --set flag-default=1')
        else: textsortcmd.append(f'--edit track:s{textid} --set flag-default=0')
#         textsortcmd+=f'--edit track:s{textid} --set flag-default=0 '
#         textsortcmd.append(f'--edit track:s{textid} --set track-number={newpos}')
        print(f'[S{textid}] {title} \t -> [S{textid}] 默认[{"*" if newpos==1 else " "}]')
        newpos+=1
    cmd=f'{os. getcwd()}\\mkvtoolnix\\mkvpropedit.exe "{filename}" {mkvpropeditcmd} '
    cmd+=" ".join(textsortcmd)
    print("================")
    print(cmd)
    return cmd

In [58]:
cmd=improve_subtitle(x264_filename)
subprocess.run(f"start cmd /c pause ^& {cmd} ^& pause", shell=True)

即将执行:
C:\Users\heber\Documents\Compressor\A Gentleman.2017.1080p.BluRay.x264.DTS-louco@PTH.mkv
[S4] chs&eng 	 -> [S4] 默认[*]
[S5] chs 	 -> [S5] 默认[ ]
[S2] cht&eng 	 -> [S2] 默认[ ]
[S3] cht 	 -> [S3] 默认[ ]
[S1] English 	 -> [S1] 默认[ ]
C:\Users\heber\Documents\Compressor\mkvtoolnix\mkvpropedit.exe "C:\Users\heber\Documents\Compressor\A Gentleman.2017.1080p.BluRay.x264.DTS-louco@PTH.mkv"  --edit track:s4 --set flag-default=1 --edit track:s5 --set flag-default=0 --edit track:s2 --set flag-default=0 --edit track:s3 --set flag-default=0 --edit track:s1 --set flag-default=0


CompletedProcess(args='start cmd /c pause ^& C:\\Users\\heber\\Documents\\Compressor\\mkvtoolnix\\mkvpropedit.exe "C:\\Users\\heber\\Documents\\Compressor\\A Gentleman.2017.1080p.BluRay.x264.DTS-louco@PTH.mkv"  --edit track:s4 --set flag-default=1 --edit track:s5 --set flag-default=0 --edit track:s2 --set flag-default=0 --edit track:s3 --set flag-default=0 --edit track:s1 --set flag-default=0 ^& pause', returncode=0)

In [59]:
cmd=improve_subtitle(x265_filename)
subprocess.run(f"start cmd /c pause ^& {cmd} ^& pause", shell=True)

即将执行:
C:\Users\heber\Documents\Compressor\A Gentleman.2017.1080p.BluRay.x265.10bit.DTS-louco@PTH.mkv
[S4] chs&eng 	 -> [S4] 默认[*]
[S5] chs 	 -> [S5] 默认[ ]
[S2] cht&eng 	 -> [S2] 默认[ ]
[S3] cht 	 -> [S3] 默认[ ]
[S1] English 	 -> [S1] 默认[ ]
C:\Users\heber\Documents\Compressor\mkvtoolnix\mkvpropedit.exe "C:\Users\heber\Documents\Compressor\A Gentleman.2017.1080p.BluRay.x265.10bit.DTS-louco@PTH.mkv"  --edit track:s4 --set flag-default=1 --edit track:s5 --set flag-default=0 --edit track:s2 --set flag-default=0 --edit track:s3 --set flag-default=0 --edit track:s1 --set flag-default=0


CompletedProcess(args='start cmd /c pause ^& C:\\Users\\heber\\Documents\\Compressor\\mkvtoolnix\\mkvpropedit.exe "C:\\Users\\heber\\Documents\\Compressor\\A Gentleman.2017.1080p.BluRay.x265.10bit.DTS-louco@PTH.mkv"  --edit track:s4 --set flag-default=1 --edit track:s5 --set flag-default=0 --edit track:s2 --set flag-default=0 --edit track:s3 --set flag-default=0 --edit track:s1 --set flag-default=0 ^& pause', returncode=0)

## 获取新文件的信息

In [33]:
generate_videoinfo(x264_filename_new,"x264.log")

NameError: name 'x264_filename_new' is not defined

In [612]:
get_mediainfo(x264_filename_new)

Copy Mediainfo


In [609]:
generate_videoinfo(x265_filename_new,"x265.log")

[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: La.grande.bouffe.1973.1080p.Blu-ray.x265.10bit.FLAC.1.0-PTH
RELEASE DATE..............: 2020-11-29
RELEASE SiZE..............: 8.786 GB
SOURCE....................: La.grande.bouffe.1973.1080p.Blu-ray.REMUX.AVC.LPCM.1.0-PTH

.Media.Info
RUNTiME...................: 2 h 9 min
ViDEO CODEC...............: x265 Main 10@L5.1@High @ 8.59Mb/s
RESOLUTiON................: 1800x1080
DiSPLAY ASPECT RATiO......: 1.667
FRAME RATE................: 23.976
BIT DEPTH.................: 10 bits
AUDiO CODEC...............: French FLAC 1 @ 637.03Kb/s 
SUBTiTLES.................: S_HDMV/PGS : English(en)
SUBTiTLES.................: S_HDMV/PGS : Chinese(CHS)
SUBTiTLES.................: S_HDMV/PGS : Chinese(CHT)


.x265.Info
x265 [info]: Main 10 profile, Level-5.1 (High tier)
x265 [info]: frame I:    916, Avg QP:20.38  kb/s: 25581.97
x265 [info]: frame P:  28949, Avg QP:21.29  kb/s: 20294.95
x265 [info]: frame

In [610]:
get_mediainfo(x265_filename_new)

Copy Mediainfo


In [82]:
def get_bbcode(file, src="",logname=""):
    print("Title:", file.replace("."," "))
    
    picbed='''
    [img]https://img.dragonhd.xyz/images/2020/12/15/7b6d5a8e1ed37ac1e687b8e3f87dd67a.jpg[/img]
    [img]https://img.dragonhd.xyz/images/2020/12/15/6cf3c3885df3138f4893c4f19ea3af45.jpg[/img]
    [img]https://img.dragonhd.xyz/images/2020/12/15/9553c945b832a40cd575eb02af548d40.jpg[/img]
    [img]https://img.dragonhd.xyz/images/2020/12/15/cb6612f30dedc0a000b745f1d06f6e29.jpg[/img]
    [img]https://img.dragonhd.xyz/images/2020/12/15/c99c7a7b54e99db2283a97b9e1d42749.jpg[/img]
    [img]https://img.dragonhd.xyz/images/2020/12/15/226756933255b37b1e656fd72dcd75a8.jpg[/img]
    [img]https://img.dragonhd.xyz/images/2020/12/15/9c6e4ac87dfe5f6075f6079bb725ad07.jpg[/img]
    '''
    if "REMUX" in file:
        picbed='''
    [img]https://cdn.longdoer.com/2020/11/30/REMUX_capture_1257be8ce9d7a6f27c.jpg[/img]
    [img]https://cdn.longdoer.com/2020/11/30/REMUX_capture_8e879ff3e0c3fc1af.jpg[/img]
    [img]https://cdn.longdoer.com/2020/11/30/REMUX_capture_3f9b9b25794869e86.jpg[/img]
    [img]https://cdn.longdoer.com/2020/11/30/REMUX_capture_50aca2c1e4633b49f.jpg[/img]
    [img]https://cdn.longdoer.com/2020/11/30/REMUX_capture_1007a6d22651cbe258.jpg[/img]
    '''
    bbcode='''
[quote]
制作说明：
原盘来自@DRs，字幕素材来自字幕库
由柠檬做柠檬水@PTHome字幕组完成校对、调整。
字幕转载务必注意礼节，请保留原作者信息，谢绝二次提取修改！
在此感谢各位原作者及分享者！如有侵权请联系删除！
[/quote]
[img]https://img9.doubanio.com/view/photo/l_ratio_poster/public/p2497633053.jpg[/img]
◎译　　名　绅士特工 / A Gentleman / Not Bang Bang 2!
◎片　　名　Reloaded
◎年　　代　2017
◎产　　地　印度
◎类　　别　喜剧 / 动作 / 爱情
◎语　　言　印地语
◎上映日期　2017-08-25(印度)
◎IMDb评分  6.2/10 from 4725 users
◎IMDb链接  https://www.imdb.com/title/tt5613834
◎豆瓣评分　5.8/10 from 558 users
◎豆瓣链接　https://movie.douban.com/subject/26904129/
◎片　　长　145分钟
◎导　　演　克里希纳·D·K Krishna D.K. / 拉杰·尼迪莫卢 Raj Nidimoru
◎编　　剧　拉杰·尼迪莫卢 Raj Nidimoru / 克里希纳·D·K Krishna D.K. / 西塔·梅诺 Sita Menon / 萨米特·巴特加 Sumit Batheja
◎主　　演　施坦·马洛萨 Sidharth Malhotra
　　　　　  杰奎琳·费南德斯 Jaqueline Fernandes
　　　　　  达山·库马尔 Darshan Kumar
　　　　　  侯赛因·达拉尔 Hussain Dalal
　　　　　  阿米特·米斯特里 Amit Mistry
　　　　　  拉吉特·卡普尔 Rajit Kapoor
　　　　　  苏普丽雅·皮尔加卡尔 Supriya Pilgaonkar
　　　　　  桑尼尔·谢迪 Sunil Shetty
　　　　　  沙希德·伍兹 Shaheed K. Woods
　　　　　  扎卡里·柯芬 Zachary Coffin
　　　　　  库沙·庞加比 Kushal Punjabi


◎标　　签　印度 | 动作 | 喜剧 | 爱情 | 2017 | 特工 | 绅士特工 | 宝莱坞

◎简　　介  

　　格拉夫是个五好青年，在迈阿密有工作有房有车，就差个女朋友。而他心仪的女孩卡维雅却觉得这个男孩太过于安全，不是自己喜欢的类型。
　　格拉夫有个在特工身份叫瑞希，在X部队执行任务。从小没有父母的他渴望拥有一个家庭，无奈头儿不让他离开这个组织。最终经过抵抗，头儿同意让他执行完最后一次任务就可以离开，过他的人生。
　　在这边，格拉夫的老板也让他去印度完成一笔交易，故事会往哪个方向发展呢？瑞希能不能得到梦想人生？格拉夫能不能得到卡维雅的芳心？
'''+f'{generate_videoinfo(file,logname,src=src)}\n{picbed}\n{get_mediainfo(file)}'
    pyperclip.copy(bbcode)

In [83]:
REMUX=r"C:\Users\heber\Music\A Gentleman.2017.1080p.BluRay.REMUX.AVC.DTS.MA.5.1-louco@PTH\A Gentleman.2017.1080p.BluRay.REMUX.AVC.DTS.MA.5.1-louco@PTH.mkv"
SRC=r"A Gentleman.2017.BD50.Untouched  BluRay.ULTRA.DRs"
get_bbcode(REMUX,src=SRC)

Title: C:\Users\heber\Music\A Gentleman 2017 1080p BluRay REMUX AVC DTS MA 5 1-louco@PTH\A Gentleman 2017 1080p BluRay REMUX AVC DTS MA 5 1-louco@PTH mkv
[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: A Gentleman.2017.1080p.BluRay.REMUX.AVC.DTS.MA.5.1-louco@PTH
RELEASE DATE..............: 2020-12-15
RELEASE SiZE..............: 31.326 GB
SOURCE....................: A Gentleman.2017.BD50.Untouched  BluRay.ULTRA.DRs

.Media.Info
RUNTiME...................: 2 h 12 min
ViDEO CODEC...............: ? High@L4.1 @ 28.61Mb/s
RESOLUTiON................: 1920x1080
DiSPLAY ASPECT RATiO......: 1.778
FRAME RATE................: 24.000
BIT DEPTH.................: 8 bits
AUDiO CODEC...............: Hindi AC-3 5.1 @ 625.0Kb/s 
AUDiO CODEC...............: Hindi DTS-HD MA 5.1 @ 4067.37Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : English(en)
SUBTiTLES.................: S_TEXT/ASS : Chinese(cht&eng)
SUBTiTLES.................: S_TEXT/ASS : Chinese(

In [84]:
get_bbcode(x264_filename,src=SRC,logname="x264.log")

Title: A Gentleman 2017 1080p BluRay x264 DTS-louco@PTH mkv
[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: A Gentleman.2017.1080p.BluRay.x264.DTS-louco@PTH
RELEASE DATE..............: 2020-12-15
RELEASE SiZE..............: 14.613 GB
SOURCE....................: A Gentleman.2017.BD50.Untouched  BluRay.ULTRA.DRs

.Media.Info
RUNTiME...................: 2 h 12 min
ViDEO CODEC...............: x264 High@L4.1 @ 13.61Mb/s
RESOLUTiON................: 1920x800
DiSPLAY ASPECT RATiO......: 2.400
FRAME RATE................: 24.000
BIT DEPTH.................: 8 bits
AUDiO CODEC...............: Hindi DTS 5.1 @ 1473.63Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : English(en)
SUBTiTLES.................: S_TEXT/ASS : Chinese(cht&eng)
SUBTiTLES.................: S_TEXT/ASS : Chinese(cht)
SUBTiTLES.................: S_TEXT/ASS : Chinese(chs&eng)
SUBTiTLES.................: S_TEXT/ASS : Chinese(chs)


.x264.Info
x264 [info]: profile High, level 4.1

In [85]:
get_bbcode(x265_filename,src=SRC,logname="x265.log")

Title: A Gentleman 2017 1080p BluRay x265 10bit DTS-louco@PTH mkv
[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: A Gentleman.2017.1080p.BluRay.x265.10bit.DTS-louco@PTH
RELEASE DATE..............: 2020-12-15
RELEASE SiZE..............: 10.446 GB
SOURCE....................: A Gentleman.2017.BD50.Untouched  BluRay.ULTRA.DRs

.Media.Info
RUNTiME...................: 2 h 12 min
ViDEO CODEC...............: x265 Main 10@L5.1@High @ 9.31Mb/s
RESOLUTiON................: 1920x800
DiSPLAY ASPECT RATiO......: 2.400
FRAME RATE................: 24.000
BIT DEPTH.................: 10 bits
AUDiO CODEC...............: Hindi DTS 5.1 @ 1473.63Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : English(en)
SUBTiTLES.................: S_TEXT/ASS : Chinese(cht&eng)
SUBTiTLES.................: S_TEXT/ASS : Chinese(cht)
SUBTiTLES.................: S_TEXT/ASS : Chinese(chs&eng)
SUBTiTLES.................: S_TEXT/ASS : Chinese(chs)


.x265.Info
x265 [info]: Mai

# 备忘录

In [720]:
generate_videoinfo(r"C:\Users\heber\Music\The.Apartment.1960.Remastered.1080p.Blu-ray.REMUX.AVC.DTS-HD.5.1-louco-PTH.mkv")

[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: C:\Users\heber\Music\The.Apartment.1960.Remastered.1080p.Blu-ray.REMUX.AVC.DTS-HD.5.1-louco-PTH
RELEASE DATE..............: 2020-11-30
RELEASE SiZE..............: 28.875 GB
SOURCE....................: La.grande.bouffe.1973.1080p.Blu-ray.REMUX.AVC.LPCM.1.0-PTH

.Media.Info
RUNTiME...................: 2 h 4 min
ViDEO CODEC...............: x264 High@L4.1 @ 30.52Mb/s
RESOLUTiON................: 1920x1080
DiSPLAY ASPECT RATiO......: 1.778
FRAME RATE................: 23.976
BIT DEPTH.................: 8 bits
AUDiO CODEC...............: English DTS 5.1 @ 750.0Kb/s 
AUDiO CODEC...............: Chinese AC-3 2 @ 218.75Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : English(en)
SUBTiTLES.................: S_HDMV/PGS : Chinese(zh)
SUBTiTLES.................: S_HDMV/PGS : Chinese(zh)
SUBTiTLES.................: S_HDMV/PGS : Chinese(zh)
SUBTiTLES.................: S_HDMV/PGS : Chinese(zh)


[/quot